In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import openpyxl
import os
import time
import math
import webbrowser
import numpy as np
import re
import sys
from pandas import DataFrame
import time
from tqdm.notebook import tqdm

In [2]:
# URL 설정 및 ChromeDriver 실행
url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000213962&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%84%A0%EC%BC%80%EC%96%B4_%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1"
driver = webdriver.Chrome()
time.sleep(2)
driver.maximize_window()
driver.get(url)
time.sleep(5)

In [5]:
temp = driver.find_elements(By.XPATH, '//*[@id="gdasList"]/li[1]/div[2]/div[3]')

print(len(temp))

for i in temp:
    print(i.text)

1
시간지나면 매마르고
엄청 소량발라야 잘 펴지고 자연스러운데 이럴거면 선크림바르는 이유가뭐지..?
적어도 약지손톱한마디는 발라야 자외선 차단이 되는데
이건 그렇게 바르면 토시오.
그리고 바르면 얼굴도 엄청 답답해서 바로 씻고싶어요
한번쓰고 두번다신 손 안감
이게어떻게34000원이지?


In [6]:
import re

# CSS Selector로 리뷰 개수 가져오기
num_models_text = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').text.strip()

# 숫자만 추출
num_models = int(re.sub(r"[^\d]", "", num_models_text))  # 숫자만 남김

# 출력
print(f"모델 개수: {num_models}")

# 페이지 수 계산
num_pages = math.ceil(num_models / 10)  # 페이지당 10개로 계산
print(f"페이지 개수: {num_pages}")

모델 개수: 4849
페이지 개수: 485


In [27]:
import pandas as pd
import time
from selenium.webdriver.common.by import By

# 데이터를 저장할 리스트
review_data = []

# 상품 리뷰 크롤링 함수
def review_crawling(page_num=5):
    for current_page in range(1, page_num + 1):
        print(f"Scraping page {current_page}...")

        # 리뷰 크롤링
        # 한 페이지에 최대 10개의 리뷰
        for i in range(1, 11):
            try:
                # 리뷰 텍스트 가져오기
                review_xpath = f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner'
                txt = driver.find_element(By.CSS_SELECTOR, review_xpath).text

                # 추가 정보 가져오기
                # select1_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dt > span').text
                # select_1_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
                select_2_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dt > span').text
                select_2_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
                select_3_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dt > span').text
                select_3_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text

                # # 데이터를 딕셔너리로 저장
                # review_data.append({
                #     "리뷰": txt,
                #     "피부타입": select1_title + ": " + select_1_content,
                #     "피부고민": select_2_title + ": " + select_2_content,
                #     "자극도": select_3_title + ": " + select_3_content
                # })
                
                # 데이터를 텍스트 형식으로 저장
                review_data.append({
                    "리뷰": txt,
                    # "피부타입": select_1_content,
                    "발림성": select_2_content,
                    "자극도": select_3_content
                })
            except Exception as e:
                print(f"Error extracting review {i} on page {current_page}: {e}")

        # 다음 페이지로 이동
        # 마지막 페이지에서는 이동하지 않음
        if current_page < page_num:
            try:
                if current_page % 10 == 0:
                    # 10의 배수일 때 다음 버튼 클릭
                    next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                # 일반 페이지 이동
                else: 
                    next_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({current_page % 10 + 2})')
                next_button.click()
                # 페이지 로딩 대기
                time.sleep(2) 
            except Exception as e:
                print(f"Error moving to page {current_page + 1}: {e}")
                break

    # DataFrame 생성
    df = pd.DataFrame(review_data)
    return df

# 실행
df_review_cos1 = review_crawling(page_num=5)

# 데이터 저장
# df_review_cos1.to_csv("reviews_suncare.csv", index=False, encoding="utf-8-sig")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...


In [28]:
df_review_cos1

,리뷰,발림성,자극도
0,워터풀이라 그런지 발림성은 부드럽게 펴바를 수 있고 뻑뻑하면서 확 하얘지는 톤업크림...,아주 만족해요,자극없이 순해요
1,엄마가 피부가 좋아보인다는 칭찬 들은 날은 꼭 거 바른 날이였다고 좋아하셔서 구매했...,아주 만족해요,보통이에요
2,자연스럽게 톤업 잘 됩니다~~\n\n가볍게 외출할 때 바르는 제품이에요.\n\n무기...,아주 만족해요,자극없이 순해요
3,톤업되면서 답답하지 않은 선크림이라 좋아요~! 톤업 제품 중에 너무 부담스럽게 톤업...,아주 만족해요,자극없이 순해요
4,일단 유분이 많고 백탁 현상이 있어요 눈에 들어가면 시립니다 톤 보정 효과는 있지만...,보통이에요,보통이에요
5,테스트 해보니 제일 좋았어요\n아침에 제대로 얼굴에 적용해봐야 알겠네요\n며칠 좀 ...,아주 만족해요,보통이에요
6,톤업듀오라 언니랑 나눴네요 일석이조 ㅋㅋㅋ 너무 좋아요 커버도 있으면서 아주 생기있...,아주 만족해요,자극없이 순해요
7,<한달 사용후기>\n\n이 제품은 사용하고나면 추가 제품 사용하지 않아도 될 정도로...,아주 만족해요,자극없이 순해요
8,달바 송민호 광고 이후로\n\n호감만 가지고 있었는데 썬크림 구입해보니\n\n순하고...,아주 만족해요,자극없이 순해요
9,"지복합성인 제가 쓸 건 아니고,,,\n아는 동생이 필요하대서 대신 구입해줬어요 ㅎㅎ...",아주 만족해요,자극없이 순해요


In [10]:
df_review_cos1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   리뷰      50 non-null     object
 1   피부타입    50 non-null     object
 2   발림성     50 non-null     object
 3   자극도     50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


In [ ]:
import pandas as pd
# 불용어를 .txt 파일에서 불러오기
with open('data/stopwords.txt', 'r', encoding='cp949') as f:
    # split() 함수를 사용 ,로 구분하여 txt 파일 문자열 구분
    stopwords = set(f.read().split(","))

# 리뷰에서 불용어 제거
def remove_stopwords(review):
    if isinstance(review, str):
        # print("Original review:", review)  # review 출력
        words = review.split()
        # print("Words:", words)  # words 출력
        filtered_words = [word for word in words if word not in stopwords]
        print('filter', filtered_words)
        return ' '.join(filtered_words)
    return review

# 데이터프레임에 불용어 제거 적용
df_review_cos1['불용어 제거 리뷰'] = df_review_cos1['리뷰'].apply(remove_stopwords)

# 수정 후 출력
print(df_review_cos1['불용어 제거 리뷰'])

In [23]:
df_review_cos1

,리뷰,피부타입,발림성,자극도
0,시간지나면 매마르고\n엄청 소량발라야 잘 펴지고 자연스러운데 이럴거면 선크림바르는 ...,지성에 좋아요,다소 아쉬워요,자극이 느껴져요
1,솔직히 머가좋다는지 모르겟음..\n복합성인데\n약간 지성인 분들 사용하면 얼굴 개떡...,건성에 좋아요,보통이에요,보통이에요
2,두개라서 오래 쓸거 같네요~~ 톤업도 되고 가끔은 이것만 바를려구요,건성에 좋아요,아주 만족해요,자극없이 순해요
3,선크림은 항상 필수인데 톤업도 되면 좋아서 항상 구매해요~,건성에 좋아요,아주 만족해요,자극없이 순해요
4,부드럽게 발리긴하는데 톤업 되는지도 모르겠고 너무 많이 밀려요...토너만 바르고 바...,복합성에 좋아요,아주 만족해요,자극없이 순해요
5,✔구매이유\n- 재구매를 여러번 한 제품이에요! 재구매를 여러번 한만큼 매번 사용할...,건성에 좋아요,아주 만족해요,자극없이 순해요
6,13호인 피부톤에는 맞는데 톤만 맞음...\n홍조 안 가려짐. 모공 반만 가려짐. ...,복합성에 좋아요,보통이에요,보통이에요
7,달바 광고를 많이 봐서 썬크림 떨어진김에 구입했어요.\n일단 약간 아쉬운 점이 많아...,복합성에 좋아요,다소 아쉬워요,자극이 느껴져요
8,듀오라 두개 가격이니까~~ 비싼건 아니네요~~ 항상 쓰는제품!,건성에 좋아요,아주 만족해요,자극없이 순해요
9,건성도 사용 못합니다.\n수분광x 번들번들 기름광입니다.,건성에 좋아요,다소 아쉬워요,보통이에요


In [97]:
import pandas as pd
import re

# 특수문자, 이모티콘, 중복 공백 제거 함수
def clean_text(text):
    # 특수문자 및 이모티콘,영어 제거 (한글, 숫자)
    text = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ0-9\s]", "", text)
    # 중복 공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 데이터프레임의 모든 컬럼에 적용
for column in df_review_cos1["불용어 제거 리뷰"]:
    df_review_cos1["불용어 제거 리뷰"] = df_review_cos1["불용어 제거 리뷰"].apply(clean_text)

# 결과 출력
df_review_cos1

# 결과 저장
# 특정 컬럼만 선택
# columns_to_save = ["불용어 제거 리뷰", "피부타입", "발림성", "자극도"]
df_review_cos1["불용어 제거 리뷰"].to_csv("suncare4_reviews_with_details.csv", index=False, encoding="utf-8-sig")

In [98]:
df_review_cos1

,리뷰,불용어 제거 리뷰
0,여름에 필수템이라 항상 구매하는데 좀 헤퍼요 ㅠㅠㅠ~ 기획아님 .. 음,여름에 필수템이라 항상 구매하는데 헤퍼요 ㅠㅠㅠ 기획아님 음
1,사용하다보면 헤프지만 몸이든 얼굴이든 분사하면 되니 간편해서 굿,사용하다보면 헤프지만 몸이든 얼굴이든 분사하면 되니 간편해서 굿
2,다들 선크림 얼굴에만 바를때 팔이랑 손가락 신경쓰는 사람이라 좋아요,다들 선크림 얼굴에만 바를때 팔이랑 손가락 신경쓰는 사람이라 좋아요
3,1+1기획이라 좋아요 세일도 많이 해주세요 상당히 헤퍼요ㅠ,11기획이라 좋아요 세일도 해주세요 상당히 헤퍼요ㅠ
4,야외활동할때 수시로 뿌려주는 용도로 구매했어요~~~~,야외활동할때 수시로 뿌려주는 용도로 구매했어요
5,급하게 뿌릴대도 좋고 외출전 마지막에 뿌리기도 좋고 휴대용도 나옴 좋겠음,급하게 뿌릴대도 좋고 외출전 마지막에 뿌리기도 좋고 휴대용도 나옴 좋겠음
6,😰 이 제품 사용후 팔에 두드러기처럼 발진 올라왔어요.... 양 팔이 전부 다요.\...,제품 사용후 팔에 두드러기처럼 발진 올라왔어요 양 팔이 다요 참고로 평소에 알러지도...
7,일반 선크림이 끈적거리고 불편해서 스프레이형을 구매하게 되었는데요.\n결과론적으로 ...,일반 선크림이 끈적거리고 불편해서 스프레이형을 구매하게 되었는데요 결과론적으로 말하...
8,매일 외출시 사용하는 꿀템🩵 (나름의 꿀팁!)\n\n특히 뜨거운 햇살에는꼭 뿌려주세...,매일 외출시 사용하는 꿀템 나름의 꿀팁 뜨거운 햇살에는꼭 뿌려주세여 선 스프레이는 ...
9,요고 분사력도 안개분사처럼 고르게 잘 분사되고 끈적임없이 약간 파우더리하게 마무리되...,요고 분사력도 안개분사처럼 고르게 잘 분사되고 끈적임없이 파우더리하게 마무리되서 여...


In [147]:
# df_review_cos1= df_review_cos1[["남길 컬럼1", "남길 컬럼2"]]
df10 = df10[["불용어 제거 리뷰", "키워드"]]
df10

,불용어 제거 리뷰,키워드
0,피부가 건조한 편인데 가을부터 예민해질 정도로 건조한 편이라 보습에 신경을 쓰려고 ...,"피부가, 건조한"
1,건성에 좋은 수분세럼이예요 이거 없음 안됨ㅜ 여름에도 겨울에도 꼭 챙겨바릅니당,"건성에, 좋은"
2,무난하게 여러통 비우고 있는 세럼이에요 블프라 세일 하길래 쟁여두려고 샀어요,"무난하게, 여러통"
3,당신이 피부든 구매 추천드립니다 주위에 있는 민감성 복합성 지성 수부지 악건성까지 ...,"당신이, 피부든"
4,수부지 피부라서 수분을 보충해줄 용도로 세럼을 구매했습니다 끈적이지 않고 촉촉해서 ...,"수부지, 피부라서"
5,제가 1년동안 잘쓴 제품 잘 사용하는 기초 중에서도 많이써서 텅비어있네요 순하게 속...,"제가, 1년동안"
6,제가 모공이 부각되어 보이는 지성피부인데 요새 화장이 뜨고 모공이 뚜렷히 보이고 걱...,"제가, 모공이"
7,올리브영에서 인기있는 제품은 다 이유가 있다는 걸 다시한번 느꼈어요 순하고 가볍게 ...,"올리브영에서, 인기있는"
8,피부가 쓰기 괜찮은 무난한 수분 세럼이에요 벌써 재구매 몇번째인지 기억이 안나네요 ...,"피부가, 쓰기"
9,토리든 믿고 씁니다 좋아요 쟁여서 쓰는데 이번에 기획세트 좋아서 샀어여,"토리든, 믿고"


## -----------------------------------------

In [146]:
import pandas as pd
import re

# 파일 경로
file_path = 'data/serum_torridendiveinserum.csv'

# CSV 파일 불러오기
df10 = pd.read_csv(file_path)

# 명사 추출 함수: 정규식을 활용해 단순히 단어를 추출
def extract_nouns_keywords(review):
    words = re.findall(r'\b\w{2,}\b', review)  # 최소 두 글자 이상의 단어 추출
    if len(words) < 3:
        return ', '.join(words)  # 단어가 3개 미만이면 그대로 반환
    selected_words = words[:2]  # 상위 3개 단어 선택
    return ', '.join(selected_words)

# '키워드' 컬럼 생성
df10['키워드'] = df10['불용어 제거 리뷰'].astype(str).apply(extract_nouns_keywords)

# 결과 확인
df10





,불용어 제거 리뷰,피부타입,피부고민,자극도,키워드
0,피부가 건조한 편인데 가을부터 예민해질 정도로 건조한 편이라 보습에 신경을 쓰려고 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,"피부가, 건조한"
1,건성에 좋은 수분세럼이예요 이거 없음 안됨ㅜ 여름에도 겨울에도 꼭 챙겨바릅니당,건성에 좋아요,보습에 좋아요,자극없이 순해요,"건성에, 좋은"
2,무난하게 여러통 비우고 있는 세럼이에요 블프라 세일 하길래 쟁여두려고 샀어요,복합성에 좋아요,진정에 좋아요,자극없이 순해요,"무난하게, 여러통"
3,당신이 피부든 구매 추천드립니다 주위에 있는 민감성 복합성 지성 수부지 악건성까지 ...,복합성에 좋아요,보습에 좋아요,보통이에요,"당신이, 피부든"
4,수부지 피부라서 수분을 보충해줄 용도로 세럼을 구매했습니다 끈적이지 않고 촉촉해서 ...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,"수부지, 피부라서"
5,제가 1년동안 잘쓴 제품 잘 사용하는 기초 중에서도 많이써서 텅비어있네요 순하게 속...,건성에 좋아요,보습에 좋아요,자극없이 순해요,"제가, 1년동안"
6,제가 모공이 부각되어 보이는 지성피부인데 요새 화장이 뜨고 모공이 뚜렷히 보이고 걱...,지성에 좋아요,보습에 좋아요,자극없이 순해요,"제가, 모공이"
7,올리브영에서 인기있는 제품은 다 이유가 있다는 걸 다시한번 느꼈어요 순하고 가볍게 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,"올리브영에서, 인기있는"
8,피부가 쓰기 괜찮은 무난한 수분 세럼이에요 벌써 재구매 몇번째인지 기억이 안나네요 ...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,"피부가, 쓰기"
9,토리든 믿고 씁니다 좋아요 쟁여서 쓰는데 이번에 기획세트 좋아서 샀어여,건성에 좋아요,보습에 좋아요,자극없이 순해요,"토리든, 믿고"


In [150]:
product_name = "수분, 진정"  # 예시 제품명
df10['기능'] = product_name

## 불용어 제거 리뷰 키워드 제품명 피부타입 기능

In [ ]:
# df1.to_csv("suncare_review.csv", index=False, encoding="utf-8-sig")
# df1 AHC
# df2 아떼
# df3 달마
# df4 식물나라
# df5 자작나무

# df6 비타민23
# df7 비타민c
# df8 아이오페
# df9 넘버즈
# df10 토리든

In [ ]:
df10

,불용어 제거 리뷰,키워드,제품명,피부타입,기능
0,저는 제품을 년째 사용 하고있는제 모르겠어요 좋은 제품이에요 자극없이 순해요 순한제...,"저는, 제품을",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
1,거짓말이 아니라 이거 몇년 쓰면서 5통 은 더비운거같은데 예민피부도 잘 쓰실거같고 ...,"거짓말이, 아니라",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
2,넘버즈인 오픈하자마자 홈페이지에서 산 사람인데 3번 결세럼 변함 예전에는 더 콧물제...,"넘버즈인, 오픈하자마자",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
3,모공피부결에 효과있는진 모르겠지만 피부에 발랐을때 느낌이 나쁘지 않은것 같습니다 정...,"모공피부결에, 효과있는진",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
4,퍼스널 컬러 겨울 클리어 봄영향을 받은 겨울클리어 타입 바우만 피부 타입 지성심한 ...,"퍼스널, 컬러",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
5,유명한 이유가 있어요사용한 직후도 그렇고 다음날 보들보들 피부장벽에도 좋고 피부결이...,"유명한, 이유가",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
6,단연코 환절기에 쓰기 좋은 세럼입니다 늘어진 모공각질 케어에 너무 좋은 세럼이에요 ...,"단연코, 환절기에",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
7,피부결에 좋다고 해서 구매했는데 사실 꾸준히 써봐도 큰 차이는 못 느끼겠어요 넘버즈...,"피부결에, 좋다고",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
8,한달사용후기 화잘먹 세럼으로 통 째 사용중 화잘먹 제가 세럼을 처음 사용해봤을 신세...,"한달사용후기, 화잘먹",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"
9,피부 타입 복합성존 지성 볼 건성 날씨가 추워지면서 여름 스킨케어 루틴만으로는 볼 ...,"피부, 타입",넘버즈인 3번 보들보들 결 세럼,복합성,"모공, 다크서클"


In [151]:
# 여러 데이터프레임 결합 예시
df_combined = pd.concat([df6, df7, df8, df9, df10], ignore_index=True)
# 결합된 데이터프레임 저장
df_combined.to_csv("serum_review.csv", index=False, encoding="utf-8-sig")